    (1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему

В данной домашней работе я обращаюсь к датасету неформальных текстовых сообщений из vk и телеграма, который был создан с моим участием. Описание идеи датасета и этапов его создания в том числе [здесь](http://www.dialog-21.ru/media/4917/babiias-kazyulinams.pdf).

**!В этом датасете может содержаться обсценная лексика и другие табуированные слова!**

Он содержит в себе сразу несколько потенциальных трудностей для автоматической частеречной разметки. Во-первых, конечно, систематические опечатки, что свойственно жанру интернет-коммуникации в целом. Во-вторых, определеная степень неграмотности. В разговорной форме письменной речи часто стираются какие-то чисто орфографические границы между разными классами e.g. путаются глаголы разных видов из-за омонимичных окончаний -тся/-ться. В-третьих, большое количество слов, которых нет в словаре: это и какие-то сленгизмы, и окказионализмы, и - в большой степени - русификация английских слов (e.g. фак). Наконец, в этом датасете также зафиксировано много оригинальных междометий, к которым относятся сразу несколько вариантов обозначения смеха.

Иными словами, тут действительно есть много специфичного материала, который возможно правильно обработать исключительно при дополнительном обучении на соответствующем дискурсе. 

In [2]:
import pandas as pd
import numpy as np 

In [3]:
df = pd.read_csv('/content/drive/MyDrive/data/validation.csv')
df.head()

,message,labels,nm_message
0,влад спасибо ещё раз,0,влад спасибо еще раз
1,а ты хоть бы привет сказал,1,а ты хоть бы привет сказал
2,аааааа что за милота,0,а что за милота
3,что если веганы захват всю зелень земли?,3,что если веганы захват всю зелень земли?
4,Не смогли бы записаться и уйти,4,не смогли бы записаться и уйти


В датасете все тексты уже прошли некоторую предобработку (детали в статье).

Однако некоторые знаки препинания еще присутствуют, поскольку датасет создавался для задачи определения эмоции в тексте. Нам же они не нужны. 

In [4]:
def delete_punctuation(text):
    return ''.join([letter for letter in text 
                    if letter.isalpha() or letter.isspace()])
texts = df.nm_message.apply(delete_punctuation).values

In [5]:
print(*texts[:5], sep='\n')

влад спасибо еще раз
а ты хоть бы привет сказал
а что за милота
что если веганы захват всю зелень земли
не смогли бы записаться и уйти


In [6]:
tokens = []
for text in texts:
    for word in text.split():
        tokens.append(word)
tokens = set(tokens)

In [7]:
from random import sample
print(*sample(tokens, 15))

скучаю их услвшать конца блятб туда вам глаз спрашивала уходил автаза lp биология глазкова красным


In [29]:
len(tokens)

591

Я сейчас не располагаю ресурсами для разметки сразу столького, поэтому возьмем половинку

In [8]:
golden_standard = sample(tokens, len(tokens)//2)
with open('to_annotate.txt', 'w', encoding='utf-8') as file:
    for token in golden_standard:
        file.write(token + '\n')

В данной домашке для ручной разметки мною был выбран набор тэгов, используемых в [UD](http://www.dialog-21.ru/media/4917/babiias-kazyulinams.pdf).
Во-первых, эта классификация представилась мне достаточно простой, чтобы сократить количество каких-то трудноразрешимых спорных случаев, коих в выбранном мною дискурсе очень и очень много. При этом этот набор вполне функционален и позволяет описать текст достаточно подробно. Наконец, этот же набор используется во многих state-of-the-art тэггерах, что может упростить дальнейшную работу. 

In [9]:
with open('annotated.txt', encoding='utf-8') as file:
    golden = list(map(lambda x: x.split('\t')[1], 
                      file.read().split('\n')))

In [30]:
print(*golden[:5])

ADJ ADJ VERB NOUN PRON


Некоторые решения, принятые мною во время разметки:
* к числительным отношу только настоящие числительные, а местоимения-числительные (несколько, сколько etc.) отношу к местоимениям
* местоимения-наречия отношу к наречиям (почему, зачем, тут etc.)
* смех в любом его письменном выражении - междометие
* слова этикета - тоже междометия (е.g. привет)
* русская запись английских слов имеют ту частеречную характеристику, какую имело это слово в англйиском (энд - cconj, фак - intj)
* причастия - это скорее прилагательные, чем глаголы (e.g. нарисованный)


    Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них. (2 балла)

### deep pavlov

In [ ]:
! pip install deeppavlov -q
! python -m deeppavlov install morpho_ru_syntagrus_pymorphy
! set "KERAS_BACKEND=tensorflow"

In [ ]:
from deeppavlov import build_model, configs
model_dp = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, 
                       download=True)

In [14]:
dp_take = [word.split('\t')[2] for word in model_dp(golden_standard)]

### stanza

In [15]:
! pip install stanza -q

     |████████████████████████████████| 342 kB 26.2 MB/s 


In [16]:
import stanza

stanza.download('ru')
model_sz = stanza.Pipeline('ru', verbose=False)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


2021-10-03 10:34:19 INFO: Downloading default packages for language: ru (Russian)...


2021-10-03 10:36:13 INFO: Finished downloading models and saved to /root/stanza_resources.


In [17]:
%%time

sz_take = [model_sz(word).sentences[0].words[0].pos 
           for word in golden_standard]

CPU times: user 20.4 s, sys: 3.85 ms, total: 20.4 s
Wall time: 20.7 s


### spacy

In [18]:
! pip install spacy==3.0.0 -q
! python -m spacy download ru_core_news_sm -q

     |████████████████████████████████| 12.7 MB 223 kB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 9.1 MB 43.3 MB/s 
     |████████████████████████████████| 623 kB 51.9 MB/s 
     |████████████████████████████████| 456 kB 66.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 0.17.1 requires pydantic==1.3, but you have pydantic 1.7.4 which is incompatible.
/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 17.9 MB 167 kB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages t

In [19]:
import spacy
model_sc = spacy.load('ru_core_news_sm')

In [20]:
%%time

sc_take = list(map(lambda x: model_sc(x)[0].pos_, golden_standard))

CPU times: user 1.82 s, sys: 6.86 ms, total: 1.83 s
Wall time: 1.83 s


    Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции. (2 балла) 

In [21]:
results = pd.DataFrame({
                        'token': golden_standard,
                        'deeppavlov': dp_take,
                        'stanza': sz_take,
                        'spaCy': sc_take,
                        'golden': golden
                        })

results.head(10)

,token,deeppavlov,stanza,spaCy,golden
0,уместнее,ADV,ADJ,ADV,ADJ
1,злой,ADJ,ADJ,ADJ,ADJ
2,скажи,VERB,VERB,VERB,VERB
3,сходку,NOUN,NOUN,NOUN,NOUN
4,моей,DET,DET,DET,PRON
5,шарю,VERB,NOUN,NOUN,VERB
6,нету,VERB,VERB,VERB,X
7,обществе,NOUN,NOUN,NOUN,NOUN
8,два,NUM,NUM,NUM,NUM
9,педофил,NOUN,NOUN,NOUN,NOUN


Мне повезло, что все интересные мне тэггеры используют систему тэгов UD. Поэтому мэтчить можем уже сейчас прямо так. 

Если бы я все-таки столкнулась с проблемой различных тэгов, то алгоритм решения этого вопроса был бы следующим:

* ручками посмотреть на наборы тэгов, решить, что в них равнозначно
* на основе проделанной аналитической работы сделать словарь в котором ключами были бы тэги, которые мы хотим использовать, а значениями - контейнеры с теми тэгами, которые используются вместо соответствующих желаемых
* итеративно пройти по оценкам тэггера с неугодной классификацией, и заменить каждый из его тэгов на тот под ключом которого находится контейнер содержащий в себе его

Сложность такого решения будет примерно (количество оценок) * (количество тэгов)


In [28]:
print(f"Deeppavlov's accuracy amounts to\
 {sum(results.deeppavlov == golden) / results.shape[0]}")


print(f"\nStanza's accuracy amounts to\
 {sum(results.stanza == golden) / results.shape[0]}")


print(f"\nSpacy's accuracy amounts to\
 {sum(results.spaCy == golden) / results.shape[0]}")


Deeppavlov's accuracy amounts to 0.8135593220338984

Stanza's accuracy amounts to 0.8169491525423729

Spacy's accuracy amounts to 0.7627118644067796


Иии у нас есть победитель: Stanza! Хотя ее отрыв от глубокого павлова совсем игрушечный. 

    Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд) (2-3 балла) 

Мне показалось не очень интересным делать функции, которые способны к выделению какого-то одного паттерна. Вместо этого я попыталась сделать что-то более абстрактное, что способно принимать широкий спектр паттернов и работать с ними соответственно.

Некоторые вспомогательные функции 

In [ ]:
from typing import List, Iterable

def get_ngrams(sequence: Iterable, ngram_size: int) -> List[List]:
    """
    Form a list of ngrams from a sequence
    :param sequence: initial sequence from which to draw ngrams
    :param ngram_size: level of ngram 
    :return: list of ngrams of a given size from sequence
    """
    return [sequence[index:index + ngram_size] 
            for index in range(len(sequence))]

def match_indices(sequence: Iterable[List], pattern: List) -> List[int]:
    """
    Find indices of the first elements of a matching pattern occurences
    :param sequence: initial sequence 
    :param pattern: pattern to match
    :return: list of indices that denote the first 
    element of a matching subsequence
    """
    return [index for index in range(len(sequence))
            if sequence[index] == pattern]
            

Собственно функция

In [71]:
def chunk(text: str, pattern: List, model: stanza.pipeline.core.Pipeline) -> List[str]:
    """
    Extract from text POS-patterns
    :param text: string in which to look for patterns
    :param model: stanza nlp model
    :param pattern: sequence of POS tags
    :return: list of strings that follow the given pattern
    """
    sentences = model_sz(text).sentences
    pos_tags, words = [], []
    for sentence in sentences:
        for word in sentence.words:
            words.append(word.text)
            pos_tags.append(word.pos)
    ngrams = get_ngrams(pos_tags, len(pattern))
    init_indices = match_indices(ngrams, pattern)
    return [' '.join(words[index:index + len(pattern)]) 
            for index in init_indices]
            

Тестируем

In [72]:
chunk('Я люблю огурцы. Он ест помидоры', ['VERB', 'NOUN'], model_sz)

['люблю огурцы', 'ест помидоры']

In [74]:
text = 'Многим очень нравится идея детерминированной вселенной. И в чем их упрекнуть? Все, что мы видим вокруг, по большей части можно либо интуитивно предсказать, либо точно описать математически, а все остальное – что ж, просто оно слишком сложное, чтобы его обсчитать, по крайней мере пока. Эйнштейн был убежден, что мнимая случайность таит под собой детерминистические законы, которые контролируют и позволяют предсказать все на свете. Если вы понимаете, с чего все начинается, законы физики точно скажут, чем все кончится. Кажется, детерминизм Вселенной встроен в формулы. Но если кажется, надо Богу молиться, и этот детерминизм – ложь и обман. На фундаментальном уровне Вселенная не столько сложна, сколько невероятно случайна. Радиоактивный распад, движение атомов, результаты физических экспериментов – все это определяется прихотями непредсказуемости. По сути своей Вселенная – страшный сон Эйнштейна. Может статься, случайность – это и ваш страшный сон. Люди плохо приспособлены к тому, чтобы мыслить статистически. Если шансы очевидны или речь идет о жизни и смерти, мозг подсказывает решение. «Не дергай за хвост этого тираннозавра [42] , – говорит, к примеру, мозг. – Многие пробовали, и ничем хорошим это не кончилось, так что все шансы за то, что и для тебя это ничем хорошим не кончится». С другой стороны, поезжайте в Лас-Вегас и спросите там человека, который только что проиграл 10 ставок подряд, каковы шансы, что в следующий раз он выиграет. Либо он заявит, что уж тут-то ему непременно повезет и он сорвет банк, либо – что игра сегодня не заладилась. Неважно, оптимист он или пессимист, – в обоих случаях он ошибается. Шансы выиграть в следующий раз точно такие же, как и во все предыдущие: 50 на 50.'

In [75]:
chunk(text, ['ADP', 'NOUN'], model_sz)

['на свете', 'в формулы', 'По сути', 'о жизни', 'за хвост', 'к примеру']

In [76]:
chunk(text, ['ADP', 'ADJ', 'NOUN'], model_sz)

['по большей части',
 'по крайней мере',
 'На фундаментальном уровне',
 'С другой стороны',
 'в следующий раз',
 'в следующий раз']

Вроде работает

Что касатется потенциально информативных паттернов, то это, на мой взгляд, в значительной мере зависит от задачи и от материала. 

Если мы говорим об отзывах, чья тональная окраска анализировалась в предыдущей домашке, то там представляются интересными следующие сочетания:

* прилагательные + существительные: такие комбинации позволяют поймать не только эмоцию, но и ее объект, то есть то, на что она направлена

* наречие + прилагательные + существительные: а вот такая комбинация может также уточнить степень проявления направленной на что-то эмоции

* наречия + глагол / глагол + наречия: такой паттерн может содержать информацию о действии продукта, которым посвящены отзывы, поскольку скорее всего в них будет приведена характеристика того как он справляется со своей задачей

Что касается внедрения анализа частеречных паттернов, то, я думаю, имеет смысл сделать это на этапе препроцессинга, где формируется список токенов, с которым в дальнейшем мы работаем. Мне кажется, нграммы можно и нужно рассматривать как равноправные токены наряду со словами. 

Таким образом, выглядеть это будет примерно так. 

In [ ]:
from module_with_global_config import MODEL

def preprocess(text: str, patterns: List[List] = None):
    clean = ''.join([i.lower() for i in text
                     if i.isalpha() or i.isspace()]) 
    tokens = [token.lemma_ for token in nlp(clean) if not token.is_stop 
          and token.lemma_.isalpha()] 
    if patterns:
        for pattern in pattenrs:
            ngrams = chunk(text, pattern, model=MODEL)
            tokens.extend(ngrams)
    return tokens
    
def create_frequency_dict(tokens):
    freqs = {}
    for token in tokens:
        if token not in freqs.keys():
            freqs[token] = 0
        freqs[token] += 1
    return freqs

Ну а все остальное то же.